## Chapter 2. 파이썬스러운(pythonic) 코드

In [2]:
my_numbers = (4, 5, 3, 9)

print("음수 인덱스:", my_numbers[-1], my_numbers[-3])
print("slice 1:", my_numbers[2:5])
print("slice 2:", my_numbers[:3])
print("slice 3:", my_numbers[3:])
print("slice 4:", my_numbers[::-1])

print("\n")
print("slice index:", my_numbers[1:7:2])
print("slice object:", my_numbers[slice(1, 7, 2)])

print("\n")
interval = slice(None, 3)
print("slice object vs index:", my_numbers[interval] == my_numbers[:3])

음수 인덱스: 9 5
slice 1: (3, 9)
slice 2: (4, 5, 3)
slice 3: (9,)
slice 4: (9, 3, 5, 4)


slice index: (5, 9)
slice object: (5, 9)


slice object vs index: True


In [1]:
""" list wrapping 사례 """

class Item:
    def __init__(self, *values):
        self._value = list(values)
    
    def __len__(self):
        return len(self._value)

    def __getitem__(self, index):
        return self._value.__getitem__(index)

In [2]:
item = Item(3, 4, 5)
print(len(item))
print(item[1:])
print(type(item[1:]))

3
[4, 5]
<class 'list'>


In [7]:
Item(3,4,5)

In [70]:
""" 컨텍스트 관리자를 활용한 DB Dump 사례 """

def stop_database():
    print("systemctl stop postgresql.service")

def start_database():
    print("systemctl start postgresql.service")


class DBHandler:
    def __enter__(self):
        stop_database()
        return self

    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()


def db_backup():
    print("pg_dump database")

def main():
    with DBHandler():
        db_backup()

main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [71]:
import contextlib

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()

def main():
    with db_handler():
        db_backup()

main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [72]:
import contextlib

class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()

    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()

@dbhandler_decorator()
def offline_backup():
    print("pg_dump database")

In [73]:
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60 # 하나의 밑줄

conn = Connector("postgresql://localhost")

conn._timeout = 70
print(conn.__dict__)

{'source': 'postgresql://localhost', '_timeout': 70}


In [74]:
class Connector:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60 # 두 개의 밑줄

conn = Connector("postgresql://localhost")

conn.__timeout # "접근할 수 없다"가 아닌 "존재하지 않는다"는 오류 발생

AttributeError: 'Connector' object has no attribute '__timeout'

In [75]:
conn._Connector__timeout = 80 # 이름이 바뀌어 있음
print(conn.__dict__)

{'source': 'postgresql://localhost', '_Connector__timeout': 80}


In [1]:
import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")

def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None

class User:
    def __init__(self, username):
        self.username = username
        self._email = None

    @property
    def email(self):
        return self._email

    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f"유효한 이메일이 아니므로 {new_email} 값을 사용할 수 없음")
        self._email = new_email


In [20]:
class Person:
    
    def __init__(self, name : str, age : int):
        self.name = name
        self.age = age
        
person = Person("Json", 24)

In [21]:
print(person.name, person.age)

Json 24


In [23]:
person.name = 10 # 문자열로 써야하지만 정수를 저장함
person.age = -100 # 나이는 마이너스가 없지만 마이너스 값을 저장함
print(person.name, person.age)

10 -100


### Person 클래스는 문자열을 저장하는 name이라는 필드와 정수를 저장하는 age라는 필드를 가지고 있다. 하지만 타입에 대한 강제성이 없는 파이썬은 외부에서 필드에 직접 접근하여 엉뚱한 값으로 수정할 때 방어 할 수 있는 방법이 없다.
>필드를 외부에서 직접 접근하면 객체가 `잘못된 상태`를 가질 수 있다. 그래서 getter와 setter를 사용하는 것이 좋다.

- 이와 같은 상황을 방지하기 위해 아래와 같이 name, age 필드를 직접 접근하는 대신 getter와 setter 함수를 만들어 값을 저장하거나 참조 할 때, 값에 대한 유효성을 검사할 수 있다.

In [77]:
class Person:
    
    def __init__(self, name :str, age : int):
        self.__name = name
        self.__age = age
        
    def get_name(self):
        return self.__name
    
    def set_name(self, name):
        if True != isinstance(name, str):
            raise ValueError('name should be str type')
        self.__name = name
        
    def get_age(self):    # getter of age
        return self.__age
    
    def set_age(self, age):
        if True != isinstance(age, int) or 0 > age:
            raise ValueError('age should be int type and not be negative')
        self.__age = age
        
person = Person('Json', 24)
#print(person.get_name(), person.get_age)
        

In [73]:
person.get_age()

24

In [74]:
person = Person(24,'json')

In [81]:
person.get_age()

14

In [76]:
person.get_name()

24

In [80]:
person.set_age('str')
person.get_age()

ValueError: age should be int type and not be negative

In [49]:
### 필드값을 저장할때 class로 접근하면 에러안뜸, set_age, set_name 으로 접근하면 에러

In [84]:
class Person:
    def __init__(self, name : str, age : int):
        self.__name = name
        self.__age = age
        
    def get_name(self):
        return self.__name
    
    def set_name(self, name):
        if True != isinstance(name, str):
            raise ValueError('name should be str type')            
        self.__name = name
        
    name = property(get_name, set_name) # name에 대한 property
    
    def get_age(self):
        return self.__age
    
    def set_age(self, age):
        if True != isinstance(age, int) or 0 > age:
            raise ValueError('age should be int and not be negative')
        self.__age = age
        
    age = property(get_age, set_age)    # age에 대한 property
    
person = Person('Jason', '24')

In [85]:
print(person.name, person.age)

Jason 24


In [86]:
#person.name = 10                 
#person.age = -100

In [87]:
# 15라인과 25라인에 프로퍼티를 사용한 것을 주목하자. property 의 첫 번째 인자로 필드의 값을 리턴하는 getter함수를, 두 번째 인자로 setter함수를 넘겨 주었고, 각각 name과 age 라는 property 객체를 만들어 저장했다.
# 위 예제에서는 getter, setter 함수가 아닌 일반 필드에 접근하는 것과 같이 name, age를 사용했지만 내부적으로 set_name, set_age를 통해 값에 대한 유효성 검증을 하는 것을 볼 수 있다.
# 만일 getter가 필요 없거나 setter가 필요없는 경우에는 아래 처럼 프로퍼티 객체 생성시 인자를 조절해 주면 된다.

In [89]:
# name = property(get_name, set_name) getter, setter 둘다 필요한 경우
# name = property(get_name) getter 만 필요한 경우
# name = property(None, set_name) setter 만 필요한 경우

### @property 데코레이터

property 클래스를 이용해 getter, setter를 일반 필드 처럼 사용 할 수 있다는 것을 앞에서 살펴 보았다. 지금 부터는 데코레이터를 이용하는 방법에 대해 살펴보도록 한다. 기능은 property 클래스와 동일하지만 데코레이터를 이용하면 데코레이터 자체가 함수의 주석과 같은 역할을 하며 코드의 가독성이 좀더 높아지게 된다. 하지만 이 부분은 개인적 취향에 따라 다르므로 그냥 데코레이터를 이용하는 방법도 있다는 것에 주목하자.

In [114]:
class Person:
    
    def __init__(self, name : str, age : int):
        self.__name = name
        self.__age = age
        
    @property
    def name(self):
        return self.__name
    
    @name.setter
    def name(self, name):
        if True != isinstance(name, str):
            raise ValueError('name should be str types')
        self.__name = name
        
    @property
    def age(self):
        return self.__age
    
    @age.setter
    def age(self, age):
        if True != isinstance(age, int) or 0 >age:
            raise ValueError('age should be int types')
        self.__age = age
        
person =Person('himaN', '15')

In [115]:
print(person.name, person.age)

himaN 15


In [116]:
person.name =10

ValueError: name should be str types

In [117]:
person.name = 'himan2'

In [118]:
person.age = -100

ValueError: age should be int types

In [119]:
person.age = 100

In [120]:
person.age

100

### 이터러블 객체

In [126]:
from datetime import timedelta
from datetime import date

class DateRangeIterable:
    """자체 이터레이터 메서드를 가지고 있는 이터러블"""
    
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date
    
    def __iter__(self):
        return self    # 자신이 이터러블임을 나타냄
    
    def __next__(self):
        if self._present_day >= self.end_date:
            raise StopIteration
            
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today

In [127]:
for day in DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5)):
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


In [132]:
r = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
next(r)

datetime.date(2019, 1, 1)

In [133]:
next(r)

datetime.date(2019, 1, 2)

In [134]:
next(r)

datetime.date(2019, 1, 3)

In [135]:
next(r)

datetime.date(2019, 1, 4)

In [137]:
next(r)

StopIteration: 

In [138]:
r1 = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
", ".join(map(str,r1))

'2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04'

In [139]:
max(r1)

ValueError: max() arg is an empty sequence

In [146]:
class DateRangeContainerIterable:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        
    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days=1)

In [147]:
r1 = DateRangeContainerIterable(date(2019, 1, 1), date(2019, 1, 5))
", ".join(map(str,r1))

'2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04'

In [148]:
max(r1)

datetime.date(2019, 1, 4)

In [157]:
# 제너레이터 표현식

generator_01 = (x for x in "1251251461324513512314523142341234")
print(generator_01)
for i in generator_01:
    print(i)

<generator object <genexpr> at 0x0000021441A7A5F0>
1
2
5
1
2
5
1
4
6
1
3
2
4
5
1
3
5
1
2
3
1
4
5
2
3
1
4
2
3
4
1
2
3
4


In [161]:
class DateRangeSequence:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._range = self._create_range()
        
    def _create_range(self):
        days = []
        current_day = self.start_date
        while current_day < self.end_date:
            days.append(current_day)
            current_day += timedelta(days=1)
            
        return days
    
    def __getitem__(self, day_no):
        return self._range[day_no]
    
    def __len__(self):
        return len(self._range)

In [163]:
s1 = DateRangeSequence(date(2019, 1, 1), date(2019, 1, 5))
for day in s1:
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


In [164]:
s1[0]

datetime.date(2019, 1, 1)

In [165]:
s1[3]

datetime.date(2019, 1, 4)

In [166]:
s1[-1]

datetime.date(2019, 1, 4)

### 컨테이너 객체

In [171]:
class Boundaries:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def __contains__(self, coord):
        x, y = coord
        return 0 <=x < self.width and 0 <= y < self.height
    
class Grid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.limits = Boundaries(width, height)

    def __contains__(self, coord):
        return coord in self.limits

def mark_coordiante(grid, coord):
    if coord in grid:
        print(f"{coord} is in limit")
    else:
        print(f"{coord} is out of the limit")

In [172]:
def mark_coordinate(grid, coord):
    if 0 <= coord.x < grid.width and 0 <= coord.y < grid.height:
        grid[coord] = MARKED

In [182]:
class Boundaries:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def __contains__(self, coord):
        x, y = coord
        return 0 <= x < self.width and 0 <= y < self.height
    
class Grid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.limits = Boundaries(width, height)
        
    def __contains__(self, coord):
        return coord in self.limits
    
def mark_coordinate(grid, coord):
    if coord in grid:
        print(f"{coord} is in limit")
    else:
        print(f"{coord} is out of the limit")

In [186]:
grid = Grid(4, 5)
coord = (3, 4)
mark_coordinate(grid, coord)

(3, 4) is in limit


### 객체의 동적인 속성

- 파이썬은 객체를 호출하면 객체 사전에서 를 찾아서 getattribute를 호출
- 객체에 찾고 있는 속성이 없는 경우 객체 이름을 파라미터로 getattr 이라는 추가 메서드가 호출됨

In [187]:
class DynamicAttributes:
    def __init__(self, attribute):
        self.attribute = attribute
        
    def __getattr__(self, attr):
        if attr.startswith("fallback_"):
            name = attr.replace("fallback_", "")
            return f"[fallback resolved] {name}"
        raise AttributeError(f"{self.__class__.__name__}에는 {attr} 속성이 없음.")

In [188]:
dyn = DynamicAttributes("value")
dyn.attribute

'value'

In [189]:
dyn.fallback_test

'[fallback resolved] test'

In [190]:
dyn.__dict__["fallback_new"] = "new value"
dyn.fallback_new

'new value'

In [192]:
getattr(dyn, 'something', "default")

'default'

In [193]:
dyn.something

AttributeError: DynamicAttributes에는 something 속성이 없음.

### 호출형(callable) 객체
- call을 사용하면 객체를 일반 함수처럼 호출할 수 있음
- 여기에 전달된 모든 파라미터는 call 메서드에 그대로 전달됨

In [196]:
from collections import defaultdict

class CallCount:
    
    def __init__(self):
        self._counts = defaultdict(int)
        
    def __call__(self, argument):
        self._counts[argument] += 1

        return self._counts[argument]

In [203]:
cc = CallCount()
print(cc(1))
print(cc(2))
print(cc(2))
print(cc(1))
print(cc(1))
print(cc(1))
print(cc(1))
print(cc(1))

1
1
2
2
3
4
5
6


### 파이썬에서 유의할 점
#### 변경 가능한 파라미터의 기본 값
- 변경 가능한 객체를 함수의 기본 인자로 사용하면 안됨

In [204]:
def wrong_user_display(user_metadata: dict = {"name":"John", "age":30}):
    name = user_metadata.pop("name")
    age = user_metadata.pop("age")
    return f"{name} ({age})"

In [205]:
wrong_user_display()

'John (30)'

In [206]:
wrong_user_display({"name":"kang", "age":20})

'kang (20)'

In [207]:
wrong_user_display()

KeyError: 'name'

- 함수 호출시 생성된 객체 메모리에 할당된 key값을 지워버렸기 때문에 에러발생

In [208]:
def good_user_display(user = None):
    user_metadata = {"name":"John", "age":30} # 기본값?

    def display(user_metadata = user_metadata):
        name = user_metadata.pop("name")
        age = user_metadata.pop("age")
        return f"{name} ({age})"

    return display(user)

- None 값으로 지정해주면 해결!

### 내장(built-in) 타입 확장
- 리스트, 문자열, 사전과 같은 내장 타입을 확장하는 올바른 방법은 collections 모듈을 사용하는 것
- 내장 함수에서 override가 동작하지 않음

In [209]:
class BadList(list):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "짝수"
            
        else:
            prefix = "홀수"
        return f"[{prefix}] {value}"

In [210]:
b1 = BadList((0,1,2,3,4,5))

In [222]:
b1[0]

'[짝수] 0'

In [213]:
b1[1]

'[홀수] 1'

In [214]:
"".join(b1)

TypeError: sequence item 0: expected str instance, int found

In [220]:
list1 = ['1','2','3']

"".join(list1)

'123'

In [223]:
from collections import UserList

class GoodList(UserList):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "짝수"
            
        else:
            prefix = "홀수"
        return f"[{prefix}] {value}"

In [224]:
g1 = GoodList((0,1,2,3,4))
g1[0]

'[짝수] 0'

In [225]:
";".join(g1)

'[짝수] 0;[홀수] 1;[짝수] 2;[홀수] 3;[짝수] 4'